### Домашнее задание «Функции и работа с данными»

А.Петряшев (DS-28)

In [1]:
import pandas as pd

In [3]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [59]:
def classif_hw(rating):
    """
       - оценка 2 и меньше - низкий рейтинг
       - оценка 4 и меньше - средний рейтинг
       - оценка 4.5 и 5 - высокий рейтинг
       
       """
    if rating['rating'] <= 2:
        return 'низкий рейтинг'
    elif rating['rating'] <= 4:
        return 'средний рейтинг'
    elif rating['rating'] >= 4.5:
        return 'высокий рейтинг'

In [58]:
ratings['class'] = ratings.apply(classif_hw, axis=1)
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


решение, если находить средний рейтинг фильма

In [54]:
def classif(rating):
    if rating <= 2:
        return 'низкий рейтинг'
    elif rating < 4:
        return 'средний рейтинг'
    elif rating >= 4:
        return 'высокий рейтинг'

In [142]:
moivies_avg_ratings = ratings[['movieId', 'rating']].groupby('movieId').mean().round(1).reset_index()
moivies_avg_ratings['class'] = moivies_avg_ratings['rating'].apply(classif)
moivies_avg_ratings['class'].value_counts()

средний рейтинг    5782
высокий рейтинг    2305
низкий рейтинг      979
Name: class, dtype: int64

In [144]:
joined = moivies_avg_ratings.merge(movies, on='movieId', how='left')
joined[['title', 'rating', 'class']].sort_values('rating', ascending=False).head()

,title,rating,class
9065,The Beatles: Eight Days a Week - The Touring Y...,5.0,высокий рейтинг
8055,One Man Band (2005),5.0,высокий рейтинг
7942,Friends with Kids (2011),5.0,высокий рейтинг
7960,"Disappearance of Haruhi Suzumiya, The (Suzumiy...",5.0,высокий рейтинг
7968,Dylan Moran: Monster (2004),5.0,высокий рейтинг


Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

1
2
3
4
5
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [129]:
regions = pd.read_csv('keywords.csv')
regions.tail()

,keyword,shows
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688
99999,ггму,3630


возможно самое простое решение прописать все возможные варианты склонения названия городов, чтобы найти все запросы, для примера добавил несколько


In [276]:
geo_data = {'Центр': ['москва', 'москвы', 'тулы' 'тула', 'ярославль', 'ярославля'], 'Северо-Запад': ['петербург', 'псков', 'мурманск', 'мурманска', 'мурманске'], 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [277]:
def geo_class(word):
    keywords_list = word.split(' ')
    if any([x in geo_data['Северо-Запад'] for x in keywords_list]):
        return 'Северо-Запад'
    elif any([x in geo_data['Центр'] for x in keywords_list]):
        return 'Центр'
    elif any([x in geo_data['Дальний Восток'] for x in keywords_list]):
        return 'Дальний Восток'
    else:
        return 'undefind'

In [280]:
regions['region'] = regions['keyword'].apply(geo_class)
regions.head()

,keyword,shows,region
0,вк,64292779,undefind
1,одноклассники,63810309,undefind
2,порно,41747114,undefind
3,ютуб,39995567,undefind
4,вконтакте,21014195,undefind


In [281]:
regions['region'].value_counts()

undefind          99290
Центр               490
Северо-Запад        156
Дальний Восток       64
Name: region, dtype: int64

Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [221]:
movies_base = pd.read_csv('movies.csv')

In [164]:
import re

In [266]:
def production_year(title):
    """
    Определение года выпуска
    
    """
    
    year_list = [str(x) for x in range(1950, 2021)]
    year_pattern = r'\(\d{4}\)'
    
    title_year = re.findall(year_pattern, title)
    
    if title_year != [] and title_year[0].strip(')(') in year_list:
    
        return title_year[0].strip(')(')
    
    else:
        
        return '1900'
production_year('Dylan Moran: Monster (2004)')

'2004'

In [264]:
movies_base['year'] = movies_base['title'].apply(production_year)
movies_base[['movieId', 'title', 'year']].value_counts('year')

year
1900    469
1996    275
2000    273
2002    272
1998    272
       ... 
1956     32
1958     31
1952     27
1950     27
1951     23
Length: 68, dtype: int64

In [265]:
movies_rating = moivies_avg_ratings.merge(movies_base, on='movieId', how='left')

movies_rating[['year', 'rating']].groupby('year').mean().sort_values('rating', ascending=False).head(10)

,rating
year,
1951,3.904348
1952,3.804000
1962,3.763415
1953,3.745714
1957,3.697500
1973,3.694118
1900,3.675165
1971,3.640741
1969,3.600000
